In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb
import csv

# Load the data
df = pd.read_csv('train.csv')
df.drop_duplicates(inplace=True)

# Separate features and target
X = df.drop('price_doc', axis=1)
y = df['price_doc']

# Convert object columns to numeric using a simpler approach
X = pd.get_dummies(X, drop_first=True)

# Log-transform numeric columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
X[num_cols] = np.log1p(X[num_cols])

# Feature selection
num_features = 70
selector = SelectKBest(f_regression, k=num_features)
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# XGBoost model
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                           max_depth=5, alpha=0.1, n_estimators=500)
xg_reg.fit(X_train, y_train)

# Predictions on the test set
y_pred = xg_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Load and preprocess the test data
df_test = pd.read_csv('test.csv')
df_test = df_test.drop(['row ID'], axis=1)

# Convert object columns to numeric using the same approach as in the training set
df_test = pd.get_dummies(df_test, drop_first=True)

# Log-transform numeric columns
df_test[num_cols] = np.log1p(df_test[num_cols])

# Apply the same feature selection to the test set
df_test_selected = df_test.iloc[:, selected_feature_indices[:len(df_test.columns)]]

# Make predictions on the test set
y_pred_test = xg_reg.predict(df_test_selected)

# Write predictions to a CSV file
filepath = 'regressor_3dec_1.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    c = 1
    writer.writerow(['row ID', 'price_doc'])
    for prediction in y_pred_test:
        writer.writerow([c, prediction])
        c += 1


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Irtiza\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Irtiza\AppData\Local\Temp\ipykernel_13864\1700050777.py", line 19, in <module>
    X = pd.get_dummies(X, drop_first=True)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Irtiza\AppData\Roaming\Python\Python311\site-packages\pandas\core\reshape\encoding.py", line 209, in get_dummies
    dummy = _get_dummies_1d(
            ^^^^^^^^^^^^^^^^
  File "C:\Users\Irtiza\AppData\Roaming\Python\Python311\site-packages\pandas\core\reshape\encoding.py", line 245, in _get_dummies_1d
    codes, levels = factorize_from_iterable(Series(data, copy=False))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Irtiza\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\categorical.py", line 2996, in factorize_from_it